## SRJ - FeS

In [56]:
## Usage of basis set
import basis_set_exchange as bse
print(bse.get_all_basis_names())

basis = bse.get_basis('ANO-RCC-MB', fmt='nwchem')
#print(basis)

import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import itertools
import time
import re
import scipy
from scipy.optimize import minimize

#qml.data.load("qchem", molname = "FeS", basis = basis)

n_basis = qml.qchem.mol_basis_data('ANO-RCC-MB', ['Fe', 'S'], load_data=basis)
print(n_basis)


['2ZaP', '2ZaPa-NR', '2ZaPa-NR-CV', '3-21G', '3ZaP', '3ZaPa-NR', '3ZaPa-NR-CV', '4-31G', '4ZaP', '4ZaPa-NR', '4ZaPa-NR-CV', '5-21G', '5ZaP', '5ZaPa-NR', '5ZaPa-NR-CV', '6-21G', '6-31++G', '6-31++G*', '6-31++G**', '6-31++G**-J', '6-31+G', '6-31+G*', '6-31+G**', '6-31+G*-J', '6-311++G', '6-311++G(2d,2p)', '6-311++G(3df,3pd)', '6-311++G*', '6-311++G**', '6-311++G**-J', '6-311+G', '6-311+G(2d,p)', '6-311+G*', '6-311+G**', '6-311+G*-J', '6-311G', '6-311G(2df,2pd)', '6-311G(d,p)', '6-311G*', '6-311G**', '6-311G**-RIFIT', '6-311G-J', '6-311xxG(d,p)', '6-31G', '6-31G(2df,p)', '6-31G(3df,3pd)', '6-31G(d,p)', '6-31G*', '6-31G**', '6-31G**-RIFIT', '6-31G*-Blaudeau', '6-31G-Blaudeau', '6-31G-J', '6ZaP', '6ZaPa-NR', '7ZaPa-NR', 'AHGBS-5', 'AHGBS-7', 'AHGBS-9', 'AHGBSP1-5', 'AHGBSP1-7', 'AHGBSP1-9', 'AHGBSP2-5', 'AHGBSP2-7', 'AHGBSP2-9', 'AHGBSP3-5', 'AHGBSP3-7', 'AHGBSP3-9', 'ANO-DK3', 'ANO-R', 'ANO-R0', 'ANO-R1', 'ANO-R2', 'ANO-R3', 'ANO-RCC', 'ANO-RCC-MB', 'ANO-RCC-VDZ', 'ANO-RCC-VDZP', 'ANO-RCC-

In [10]:
n_basis

([19, 9],
 (((0, 0, 0),
   [4316265.0,
    646342.4,
    147089.7,
    41661.52,
    13590.77,
    4905.75,
    1912.746,
    792.6043,
    344.8065,
    155.8999,
    72.23091,
    32.72506,
    15.66762,
    7.503483,
    3.312223,
    1.558471,
    0.683914,
    0.146757,
    0.070583,
    0.031449,
    0.01258],
   [0.00015003,
    0.00043597,
    0.00120365,
    0.00312635,
    0.00814591,
    0.02133892,
    0.05470838,
    0.12845394,
    0.25203824,
    0.35484986,
    0.27043078,
    0.06476086,
    -0.00110466,
    0.00184555,
    -0.000856,
    0.00037119,
    -0.00014687,
    6.097e-05,
    -5.789e-05,
    2.77e-05,
    -7.22e-06]),
  ((0, 0, 0),
   [4316265.0,
    646342.4,
    147089.7,
    41661.52,
    13590.77,
    4905.75,
    1912.746,
    792.6043,
    344.8065,
    155.8999,
    72.23091,
    32.72506,
    15.66762,
    7.503483,
    3.312223,
    1.558471,
    0.683914,
    0.146757,
    0.070583,
    0.031449,
    0.01258],
   [-4.622e-05,
    -0.00013445,
    -0

In [13]:
#Check the possible excitations
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import itertools
import time
import re
import scipy
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []
old_grad = []
excitationlist = []
generatingfns = []
gs_energy = []
theta = np.pi/4
operator_check = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity



bond_length = 1.826
symbols = ["Fe","S"]
electrons = 4
orbitals = 12
r_bohr = bond_length *1.8897259886  
coordinates = np.array([[0.0,0.0, 0*r_bohr], [0.0, 0.0, 1*r_bohr]])
molecule = qml.qchem.Molecule(symbols, coordinates, basis_name='ano-rcc-mb')


H, qubits = qml.qchem.molecular_hamiltonian(molecule)
#hf_state = qchem.hf_state(electrons, qubits)

def adaptvqe(adapt_it = 4, e_th=1e-12):
    ash_excitation = []
    energies = []
    excitations= []
    #Preparation of the Hartree-Fock state in the form of 2^8 states
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def hf_stateprep(wires):
        target_state = np.zeros(2**qubits)
        target_state[3840] = 1.0 # Every molecule change, you need to change this index #3840
        qml.StatePrep(target_state, wires=range(qubits))
        return qml.state()

    hf_state = hf_stateprep(wires=qubits)
    print('HF state is', hf_state)

    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, electrons, qubits, H):
        #print('Updated hf_state is', hf_state)  
        qml.StatePrep(hf_state, wires=range(qubits))
        return qml.expval(H)  
    
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the ash_excitationis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)
    #Applyign the givens rotation to the HF state
    #For generating the new state, they are adding the excitation in the exponent of the rotation, thats 
    #why we need to use qml.FermionicSE or qml.FermionicDE
    @qml.qnode(dev)
    def new_state(hf_state, ash_excitation, qubits):
        qml.StatePrep(hf_state, wires=range(qubits))
        for i in range(len(ash_excitation)):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=theta, wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=theta, wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
     
    #Measurement of new state
    dev_meas = qml.device("lightning.qubit", wires=qubits, shots=1000)
    @qml.qnode(dev_meas)
    def measure(ostate):
        qml.StatePrep(ostate, wires=range(qubits))
        return qml.counts()
    
    @qml.qnode(dev)
    def ind_state(ash_excitation):
        qml.StatePrep(hf_state, wires=range(qubits))
        print('Individual excitation is', ash_excitation)
        if len(ash_excitation) == 4 :
            qml.DoubleExcitation(theta, wires=ash_excitation)
        elif len(ash_excitation) == 2:
            qml.SingleExcitation(theta, wires=ash_excitation)
        return qml.state()


    print('HF state is', circuit(hf_state, electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(electrons, orbitals)

            #print('D-Generating functions are', super2)
    #print('Total len of Generating functions is', len(generatingfns))
    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    states = [hf_state]
    print('The length of operator pool', len(operator_pool))


    for j in range(1, adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
        
        
        for i in operator_pool:
            if i not in operator_check:  # Check if the operator is already checked
            #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
                w = qml.fermi.jordan_wigner(i)  #JW transformation
                if np.array_equal(k, hf_state): # If the current state is the HF state
                    #print('Print, if this is activated - HF state')
                    current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
                else:
                    current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
                #print(f'The expectation value of {i} is', current_value)

                if current_value > max_value:
                    max_value = current_value
                    max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value
        operator_check.append(max_operator)

        # Convert operator to excitations and append to ash_excitation
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations1 = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations1)
        ash_excitation.append(excitations1)
        print('ash_excitation is', ash_excitation)
        print('The length of ash_excitation before generating matrix is', len(ash_excitation))

        
        #Now apply the givens rotation for it. 
        ostate = new_state(hf_state, ash_excitation, qubits)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation, qubits))
        #Measuring the state
        counts = measure(ostate)
        print('State count after Generating function is', counts)
        states.append(ostate)
        if j >= 2:
            states.append(ind_state(ash_excitation[-1]))  # Append the individual excitation state
        else:
            print('Skipping individual excitation state for j < 2')
        print('After adding new state, the length of states is', len(states))
        M = np.zeros((len(states), len(states)), dtype=complex)  # Initialize H with zeros
        S = np.zeros((len(states), len(states)), dtype=complex)  # Initialize S matrix with zeros
        Ham_matrix = qml.matrix(H)
        print('Ham matrix is', Ham_matrix)
        # Now we need to fill the H and S matrices
        for i in range(len(states)):
            for j in range(len(states)):
                print('The value of i and j is', i, j)
                S[i,j] = (states[i].conj().T) @ states[j] #<psi_i|psi_j>
                M[i,j] = (states[i].conj().T) @ Ham_matrix @ states[j]
                if i != j:
                    M[j,i] = M[i,j].conj()
                    S[j,i] = S[i,j].conj()
        n = S.shape[0]
        epsilon = 1e-10
        S_reg = S + epsilon * np.eye(n)
        print(f'S matrix with {i} and {j} is', S)
        print('Hamiltonian Matrix with', i, j, 'is', M)
        eig,evec=scipy.linalg.eigh(M,S_reg)
        print('Eigenvalues are', eig)
        gs_energy.append(eig[0])
        print('Ground state energy is', gs_energy)    
    return ash_excitation, states,eig,gs_energy, Ham_matrix


ValueError: Currently, the only supported basis sets are 'sto-3g', '6-31g', '6-311g' and 'cc-pvdz'. Please consider using `load_data=True` to download the basis set from an external library that can be installed with: pip install basis-set-exchange.

In [55]:
import pyscf
import basis_set_exchange as bse
#print(bse.get_all_basis_names())

basis = bse.get_basis('ANO-RCC-MB', elements = ['Fe', 'S'], fmt = 'gaussian')
print(basis)
#print(dict(basis))
#alpha = basis['exponents']
#print(alpha)

RuntimeError: Unknown basis set format "gaussian"

In [49]:
basis

{'molssi_bse_schema': {'schema_type': 'complete', 'schema_version': '0.1'},
 'revision_description': 'Data from OpenMolCAS',
 'revision_date': '2019-03-05',
 'elements': {'16': {'electron_shells': [{'function_type': 'gto',
     'region': '',
     'angular_momentum': [0],
     'exponents': ['346348.23',
      '49391.146',
      '14610.990',
      '5187.2095',
      '1980.9676',
      '784.63139',
      '317.32779',
      '130.01976',
      '53.738208',
      '22.345896',
      '9.3332512',
      '3.9111868',
      '1.6432066',
      '.69174560',
      '.29167360',
      '.12314410',
      '.04925760'],
     'coefficients': [['0.00029092',
       '0.00094665',
       '0.00211923',
       '0.00602161',
       '0.01659726',
       '0.04683989',
       '0.12242581',
       '0.27022435',
       '0.40541657',
       '0.25571527',
       '0.02671693',
       '0.00390332',
       '-0.00204189',
       '0.00149034',
       '-0.00062107',
       '0.00044167',
       '-0.00000324'],
      ['-0.000

In [53]:
alpha = np.array([[],
                [3.42525091, 0.62391373, 0.1688554]], requires_grad = True) #Exponents

KeyError: 'exponent'

In [40]:
import pennylane as qml
from pennylane import numpy as np

# Define molecule
symbols = ['Fe', 'S']
geometry = np.array([[0.0, 0.0, 0.0],
                     [0.0, 0.0, 2.0]])

basis_name = "ANO-RCC-MB"  # The name must match the BSE convention

# Load basis set data for the molecule
n_basis, basis_data = qml.qchem.mol_basis_data(basis_name, symbols, load_data=True)
#print(n_basis)
#print(basis_data)

alpha = basis_data
print(alpha)
#coeff = basis_data['coefficients']

#class Molecule(symbols, coordinates, charge=0, mult=1, basis_name='sto-3g', name='molecule', load_data=False, l=None, alpha=None, coeff=None, normalize=True, unit='bohr')
#H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, method="pyscf", alpha=alpha, coeff=coeff)
molecule = qml.qchem.Molecule(symbols, geometry, load_data = True, alpha = basis_data, coeff = basis_data['coefficients'])


H, qubits = qml.qchem.molecular_hamiltonian(molecule)
print(H)




(((0, 0, 0), [4316265.0, 646342.4, 147089.7, 41661.52, 13590.77, 4905.75, 1912.746, 792.6043, 344.8065, 155.8999, 72.23091, 32.72506, 15.66762, 7.503483, 3.312223, 1.558471, 0.683914, 0.146757, 0.070583, 0.031449, 0.01258], [0.00015003, 0.00043597, 0.00120365, 0.00312635, 0.00814591, 0.02133892, 0.05470838, 0.12845394, 0.25203824, 0.35484986, 0.27043078, 0.06476086, -0.00110466, 0.00184555, -0.000856, 0.00037119, -0.00014687, 6.097e-05, -5.789e-05, 2.77e-05, -7.22e-06]), ((0, 0, 0), [4316265.0, 646342.4, 147089.7, 41661.52, 13590.77, 4905.75, 1912.746, 792.6043, 344.8065, 155.8999, 72.23091, 32.72506, 15.66762, 7.503483, 3.312223, 1.558471, 0.683914, 0.146757, 0.070583, 0.031449, 0.01258], [-4.622e-05, -0.00013445, -0.00037184, -0.00096889, -0.00253948, -0.00673001, -0.0176816, -0.0437541, -0.09601111, -0.16998599, -0.18456376, 0.058263, 0.52163758, 0.49331062, 0.0863267, -0.00285017, 0.00165569, -0.00049176, 0.00047608, -0.0002282, 6.297e-05]), ((0, 0, 0), [4316265.0, 646342.4, 147089

TypeError: tuple indices must be integers or slices, not str

In [15]:
import pennylane as qml

n_basis, params = qml.qchem.mol_basis_data('sto-3g', ['H', 'H'])
print(n_basis)
print(params)

[1, 1]
(((0, 0, 0), [3.425250914, 0.6239137298, 0.168855404], [0.1543289673, 0.5353281423, 0.4446345422]), ((0, 0, 0), [3.425250914, 0.6239137298, 0.168855404], [0.1543289673, 0.5353281423, 0.4446345422]))


## SRJ - Present 

In [ ]:
import basis_set_exchange as bse

basis_dict = bse.get_basis('ANO-RCC-MB', elements=['Fe', 'S'], fmt='nwchem')

import numpy as np
from pyscf import gto, scf, mcscf
from pyscf import gto, ao2mo, scf
import pennylane as qml

mol_pyscf = gto.M(
    atom = 'Fe 0 0 0; S 0 0 1.826',
    basis = basis_dict,
    spin = 4)
rhf = scf.RHF(mol_pyscf)
rhf.kernel()

one_ao = mol_pyscf.intor_symmetric('int1e_kin') + mol_pyscf.intor_symmetric('int1e_nuc')
two_ao = mol_pyscf.intor('int2e_sph')

one_mo = np.einsum('pi,pq,qj->ij', rhf.mo_coeff, one_ao, rhf.mo_coeff)
two_mo = ao2mo.incore.full(two_ao, rhf.mo_coeff)

two_mo = np.swapaxes(two_mo, 1, 3)
core_constant = np.array([rhf.energy_nuc()])

H_fermionic = qml.qchem.fermionic_observable(core_constant, one_mo, two_mo)

H = qml.jordan_wigner(H_fermionic)
#print(H)


## SRJ - Hamiltonian generation successful

In [1]:
import basis_set_exchange as bse

basis_dict = bse.get_basis('ANO-RCC-MB', elements=['Fe', 'S'], fmt='nwchem')
#basis_dict = bse.get_basis('ANO-RCC-MB', fmt='nwchem')

## Working code

In [7]:
import numpy as np
from pyscf import gto, scf, mcscf
from pyscf import gto, ao2mo, scf
import pennylane as qml

# Your original setup (unchanged)
mol_pyscf = gto.M(
    atom = 'Fe 0 0 0; S 0 0 1.826',
    basis = basis_dict,
    spin = 4)
rhf = scf.UHF(mol_pyscf)
rhf.kernel()


print("Atom list:", mol_pyscf.atom)
print("Basis used:", mol_pyscf.basis)
print("Charge:", mol_pyscf.charge)
print("Spin:", mol_pyscf.spin)
print("Number of electrons:", mol_pyscf.nelectron)

# === ACTIVE SPACE IMPLEMENTATION ===
# Step 1: Define active space parameters
norb_active = 6      # Number of active orbitals
nelec_active = 6     # Number of active electrons

# Step 2: Set up CASSCF to get active space integrals
mycas = mcscf.CASSCF(rhf, norb_active, nelec_active)
mycas.kernel()  # Run CASSCF to get the ground state energy and active space integrals
print("Hartree-Fock Ground State Energy:", rhf.e_tot)
print("CASSCF Ground State Energy:", mycas.e_tot)

# Step 3: Get effective active space integrals
h1e_cas, ecore = mycas.get_h1eff()  # One-electron integrals + core energy
h2e_cas = mycas.get_h2eff()         # Two-electron integrals

print("h2e_cas type:", type(h2e_cas))
print("h2e_cas shape:", h2e_cas.shape) # This result shows a 2D array, symmetric reduced form

norb = norb_active  

# "21" = norb*(norb+1)//2 for norb=6, so s4 symmetry - It was symmetrical reduced. Got into case with no symmetry
h2e_4d = ao2mo.restore(1, h2e_cas, norb) #Check this link (https://pyscf.org/pyscf_api_docs/pyscf.ao2mo.html)




one_mo = h1e_cas
two_mo = h2e_4d
h2_phys = np.swapaxes(two_mo, 1, 3)  # Ensure the two-electron integrals are in the correct order
core_constant = np.array([ecore])

# Step 5: Continue with your original PennyLane code (unchanged)
#H_fermionic = qml.qchem.fermionic_observable(core_constant, one_mo, h2_phys)
#H = qml.jordan_wigner(H_fermionic)


converged SCF energy = -1655.1015537099  <S^2> = 6.0386361  2S+1 = 5.0154306
Atom list: Fe 0 0 0; S 0 0 1.826
Basis used: #----------------------------------------------------------------------
# Basis Set Exchange
# Version 0.11
# https://www.basissetexchange.org
#----------------------------------------------------------------------
#   Basis set: ANO-RCC-MB
# Description: ANO-RCC-MB
#        Role: orbital
#     Version: 1  (Data from OpenMolCAS)
#----------------------------------------------------------------------


BASIS "ao basis" SPHERICAL PRINT
#BASIS SET: (17s,12p) -> [3s,2p]
S    S
 346348.23                   0.00029092            -0.00008101             0.00002280
  49391.146                  0.00094665            -0.00026388             0.00007421
  14610.990                  0.00211923            -0.00059263             0.00016719
   5187.2095                 0.00602161            -0.00168762             0.00047372
   1980.9676                 0.01659726            -0.00